In [1]:
import pandas as pd

# ====== CSVのパス ======
CAT_CSV   = "csv_exports/category.csv"
FC_CSV    = "csv_exports/film_category.csv"
FILM_CSV  = "csv_exports/film.csv"
INV_CSV   = "csv_exports/inventory.csv"
RENT_CSV  = "csv_exports/rental.csv"
CUS_CSV   = "csv_exports/customer.csv"
ADDR_CSV  = "csv_exports/address.csv"
CITY_CSV  = "csv_exports/city.csv"

# ====== 都市×カテゴリの件数集計 ======
def make_city_category_counts():
    cat   = pd.read_csv(CAT_CSV)
    fc    = pd.read_csv(FC_CSV)
    film  = pd.read_csv(FILM_CSV)
    inv   = pd.read_csv(INV_CSV)
    rent  = pd.read_csv(RENT_CSV, parse_dates=["rental_date"])
    cus   = pd.read_csv(CUS_CSV)
    addr  = pd.read_csv(ADDR_CSV)
    city  = pd.read_csv(CITY_CSV)

    # 顧客→住所→都市
    cus_city = cus.merge(addr, on="address_id", how="left") \
                  .merge(city, on="city_id", how="left")

    # rental -> customer -> city
    r_cus = rent.merge(cus_city[["customer_id","city"]], on="customer_id", how="left")

    # rental -> inventory(→film) -> film_category -> category
    df = (r_cus.merge(inv[["inventory_id","film_id"]], on="inventory_id", how="left")
                 .merge(fc, on="film_id", how="left")
                 .merge(cat, on="category_id", how="left"))

    # ピボット（件数）
    pt = pd.pivot_table(df, index="city", columns="name",
                        values="rental_id", aggfunc="count", fill_value=0)

    return pt

if __name__ == "__main__":
    pt_city = make_city_category_counts()
    print("都市×カテゴリの件数ピボット（先頭5行）:")
    print(pt_city.head(), "\n")

    # CSVに保存
    out_csv = "city_category_counts.csv"
    pt_city.to_csv(out_csv, encoding="utf-8-sig")
    print(f"CSVファイルに出力しました: {out_csv}")


都市×カテゴリの件数ピボット（先頭5行）:
name                Action  Animation  Children  Classics  Comedy  \
city                                                                
A Corua (La Corua)       1          0         3         3       4   
Abha                     1          5         1         0       1   
Abu Dhabi                4          1         3         0       0   
Acua                     2          0         3         1       2   
Adana                    3          1         3         0       3   

name                Documentary  Drama  Family  Foreign  Games  Horror  Music  \
city                                                                            
A Corua (La Corua)            4      2       1        2      2       2      1   
Abha                          0      0       0        0      2       3      2   
Abu Dhabi                     1      4       3        2      2       1      1   
Acua                          2      3       3        2      2       1      1   
Adana   